In [1]:
# 列表：由其他对象的引用组成的连续数组
a = 1
b = 'hello'

c = [a, b]
c

[1, 'hello']

In [2]:
# a变成了对2的引用
a = 2

# 但列表的引用对象仍是：1， hello
c

[1, 'hello']

In [3]:
# 列表的引用对象变为：1，world
c[1] = 'world'

# 但b的引用对象仍是world
b

'hello'

In [5]:
# 列表推导式：一般而言，列表推导式比for循环跟快速。
[i for i in range(10) if i % 2 == 0]

[0, 2, 4, 6, 8]

In [6]:
# zip函数：对zip函数返回的结果再次调用zip函数，可以使其恢复原状
for item in zip(*zip([1, 2, 3], [4, 5, 6])):
    print(item)

(1, 2, 3)
(4, 5, 6)


In [8]:
# 序列解包 -- *号的用法
first, *inner, last = 0, 1, 2, 3

print('first:', first)
print('inner:', inner)
print('last:', last)

first: 0
inner: [1, 2]
last: 3


In [9]:
# 嵌套解包
(a, b), (c, d) = (1, 2), (3, 4)
a, b, c, d

(1, 2, 3, 4)

In [10]:
# dict的视图对象（view objects）: keys()、values()和items()均返回视图对象
# 字典发生变化时，视图也会发生相应的变化
# 字典在CPython中使用hash table实现，所以只有hashable objects才能作为字典的键。
# python中所有不可变类型都是可哈希的
word = {'a': 1, 'b': 2}
items = word.items()
word['c'] = 3
items

dict_items([('a', 1), ('b', 2), ('c', 3)])

In [11]:
# 迭代器：实现了迭代器协议的容器对象，包含一下2个方法
# __next__：返回容器的一下元素
# __iter__：返回迭代器本身
class CountDown:
    def __init__(self, step):
        self.step = step

    def __next__(self):
        """返回容器的下一个元素"""
        if self.step <= 0:
            raise StopIteration
        self.step -= 1
        return self.step

    def __iter__(self):
        """返回迭代器本身"""
        return self

for i in CountDown(4):
    print(i)

3
2
1
0


In [12]:
# 生成器：使用yield返回结果的一类特殊迭代器(保存执行上下文)
# 一个简单的生成器
def fibonacci():
    a, b = 0, 1

    while True:
        yield b
        a, b = b, b + a

fib = fibonacci()
next(fib)

1

In [13]:
next(fib)

1

In [14]:
next(fib)

2

In [15]:
[next(fib) for i in range(10)]

[3, 5, 8, 13, 21, 34, 55, 89, 144, 233]